In [11]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import files

uploaded = files.upload()

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("charity_data.csv")
df.head()

Saving charity_data.csv to charity_data (2).csv


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [12]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df.drop(['EIN', 'NAME'], axis=1, inplace= True)

In [13]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [14]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = ['C1256','C3200','C1267','C1246','C1234','C2710','C1260','C0','C1600','C1257','C4100','C2400','C1720','C7210','C1235','C1237','C1238','C1278','C8200','C1250','C6000','C1800',
                                'C1500,','C7120','C8000','C1240','C2300','C7200','C1400','C1230','C2300','C7200','C1400','C1230','C1280','C1300','C7100','C2800','C2700', 'C1270','C5000','C4000','C1700',
                                  'C7000','C1500','C1370','C1236','C2190','C4200','C5200','C6100','C2600','C1248','C1580','C1820','C1900','C2570','C1283','C3700','C2500','C1570','C2380','C1732','C1728',
                                    'C2170','C2170','C4120','C8210','C4500','C1245','C2561','C2150']

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [15]:
df = pd.get_dummies(df, prefix='', prefix_sep='',
                                columns= ['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS'])

In [16]:
# Split our preprocessed data into our features and target arrays
y = df["Y"].values
X = df.drop(["Y",'N'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Model 1
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 344       
                                                                 
 dense_4 (Dense)             (None, 5)                 45        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 395
Trainable params: 395
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0057 - accuracy: 0.9992
Epoch 2/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0056 - accuracy: 0.9992
Epoch 3/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0056 - accuracy: 0.9992
Epoch 4/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0055 - accuracy: 0.9992
Epoch 5/25
804/804 [==============================] - 2s 2ms/step - loss: 0.0055 - accuracy: 0.9992
Epoch 6/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0055 - accuracy: 0.9992
Epoch 7/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0054 - accuracy: 0.9992
Epoch 8/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0054 - accuracy: 0.9992
Epoch 9/25
804/804 [==============================] - 2s 3ms/step - loss: 0.0054 - accuracy: 0.9992
Epoch 10/25
804/804 [==============================] - 3s 4ms/step - loss: 0.0054 - accuracy: 0.9992

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.0050 - accuracy: 0.9991 - 436ms/epoch - 2ms/step
Loss: 0.004970563109964132, Accuracy: 0.9990670680999756


In [24]:
# Model 2 / changed the amount of hidden nodes
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  5
hidden_nodes_layer2 = 5

fn = tf.keras.models.Sequential()

# First hidden layer
fn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
fn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
fn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
fn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 5)                 215       
                                                                 
 dense_7 (Dense)             (None, 5)                 30        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
fn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = fn.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25
804/804 [==============================] - 2s 2ms/step - loss: 0.1662 - accuracy: 0.9603
Epoch 2/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0088 - accuracy: 0.9991
Epoch 3/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0066 - accuracy: 0.9991
Epoch 4/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0061 - accuracy: 0.9992
Epoch 5/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0057 - accuracy: 0.9992
Epoch 6/25
804/804 [==============================] - 2s 2ms/step - loss: 0.0056 - accuracy: 0.9992
Epoch 7/25
804/804 [==============================] - 2s 3ms/step - loss: 0.0055 - accuracy: 0.9992
Epoch 8/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0055 - accuracy: 0.9992
Epoch 9/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0055 - accuracy: 0.9992
Epoch 10/25
804/804 [==============================] - 2s 2ms/step - loss: 0.0054 - accuracy: 0.9992

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.0050 - accuracy: 0.9991 - 562ms/epoch - 2ms/step
Loss: 0.004970563109964132, Accuracy: 0.9990670680999756


In [28]:
# Model 3 / changed the amount of hidden nodes
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  7
hidden_nodes_layer2 = 7

an = tf.keras.models.Sequential()

# First hidden layer
an.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
an.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
an.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
an.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 7)                 301       
                                                                 
 dense_10 (Dense)            (None, 7)                 56        
                                                                 
 dense_11 (Dense)            (None, 1)                 8         
                                                                 
Total params: 365
Trainable params: 365
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
an.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = an.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25
804/804 [==============================] - 2s 2ms/step - loss: 0.3188 - accuracy: 0.8704
Epoch 2/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0104 - accuracy: 0.9992
Epoch 3/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0067 - accuracy: 0.9992
Epoch 4/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0061 - accuracy: 0.9992
Epoch 5/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0058 - accuracy: 0.9992
Epoch 6/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0057 - accuracy: 0.9992
Epoch 7/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0056 - accuracy: 0.9992
Epoch 8/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0056 - accuracy: 0.9992
Epoch 9/25
804/804 [==============================] - 1s 2ms/step - loss: 0.0055 - accuracy: 0.9992
Epoch 10/25
804/804 [==============================] - 2s 2ms/step - loss: 0.0054 - accuracy: 0.9992

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.0050 - accuracy: 0.9991 - 317ms/epoch - 1ms/step
Loss: 0.004970563109964132, Accuracy: 0.9990670680999756
